# Introduction
Martyn previously made a script to report the growth rate of the model when grown on a range of different sugars. As we are looking into the oxygen consumption, it would also be interesting to plot the minimum oxygen requirement when growing on these sugars. So here I will make a script to calculate the growth rate of the model when the same amount in Cmol of substratre is supplied, and accordingly compute the minimum oxygen required to achieve that growth rate.

N.B.: this is also done to check the values that Martyn gets, as we've been having different similuations...

In [1]:
import cameo
import pandas as pd
import cobra.io
import copy
import numpy as np
import matplotlib.pyplot as plt
from cobra.flux_analysis import flux_variability_analysis

In [2]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
sugars = ['arbt_e', 'cellb_e', 'malt_e', 'man_e','sucr_e','tre_e', 'tura_e', 'xyl__D_e', 'fru_e','gtbi_e', 'glc__D_e', 'glyc_e', 'arab__L_e', 'rmn_e', 'mdgp_e','rib__D_e', 'salcn_e', 'sbt__D_e', 'gal_e', 'melib_e','amylose_e', 'kdg2_e']

I will write a script that automatically sets the exchange boundary of each sugar to a supply of 30 mmolC/gcdw/h (i.e. 5 mmol glucsoe/gcdw/h). 

In [20]:
pred_growth = [] #store the predicted growths here
pred_yld = []
min_O2 = [] #store the minimal oxygen needed
with model:
    #set glc exchange to zero
    model.reactions.EX_glc__D_e.bounds = (0,1000)
    for carb in sugars:
        model_copy = copy.deepcopy(model) 
        met = model_copy.metabolites.get_by_id(carb)
        rct_id = 'EX_'+ carb
        exch = model_copy.reactions.get_by_id(rct_id)
        carbon = met.elements['C']
        lb = -30/carbon #i.e. how many mols of the sugar we would supply
        #set uptake
        exch.bounds = (lb,1000)
        #predict growth
        try: 
            growth = model_copy.optimize().objective_value
            MW = met.formula_weight/1000 #in g/mmol
            yld = (growth/-lb)/MW #in gx/gs
        except UserWarning:
            growth = 0
            yld = 0
        pred_growth.append(growth)
        pred_yld.append(yld)
        #then fix the growth rate, and minimize oxygen uptake
        model_copy.reactions.biomass.bounds = (growth,growth) #set growth rate to maximum defined above
        model_copy.objective = 'EX_o2_e'
        model_copy.objective_direction = 'min'
        try:
            o2 = model_copy.optimize()['EX_o2_e']
        except UserWarning:
            o2 = 0
        min_O2.append(-o2)    

In [5]:
model.metabolites.lac__D_c.name = 'D-Lactate'
model.metabolites.lac__L_c.name = 'L-Lactate'

In [6]:
model.metabolites.lac__D_e.name = model.metabolites.lac__D_c.name
model.metabolites.lac__L_e.name = model.metabolites.lac__L_c.name
model.metabolites.glc__D_e.name = model.metabolites.glc__D_c.name

In [7]:
model.metabolites.xyl__D_e.name = 'D-Xylose'
model.metabolites.arab__L_e.name = 'L-Arabinose'
model.metabolites.rmn_e.name = 'L-Rhamnose'


In [8]:
#make a list of more infomartive names for the sugars
sugar_name = []
for sugar in sugars:
    sugar_name.append(model.metabolites.get_by_id(sugar).name)

In [9]:
#make a dataframe of the three lists
df_aer = pd.DataFrame({'Sugar':sugar_name, 'Growth': pred_growth, 'Biomass Yield (g/g)':pred_yld, 'Oxygen': min_O2})

In [10]:
order = ['Amylose', 'L-Arabinose', 'Arbutin', 'Cellobiose','D-Fructose', 'Gentiobiose', 'D-Galactose', 'D-Glucose', 'Glycerol', '2-Keto-D-gluconic acid', 'Maltose', 'D-Mannose', 'Melibiose', 'Methyl D-glucopyranoside', 'L-Rhamnose', 'D-Ribose', 'Salicin', 'D-Sorbitol', 'Sucrose', 'Trehalose', 'D-Turanose', 'D-Xylose'] 

In [11]:
df_order = pd.DataFrame({'Sugar':order})

In [12]:
df_aer_ordered = pd.merge(df_order, df_aer, on='Sugar')

In [13]:
df_aer_ordered

,Sugar,Growth,Biomass Yield (g/g),Oxygen
0,Amylose,0.327637,0.393217,17.649437
1,L-Arabinose,0.307471,0.341339,18.409626
2,Arbutin,0.271804,0.399343,21.004132
3,Cellobiose,0.327637,0.382869,17.649437
4,D-Fructose,0.327408,0.363472,17.658075
5,Gentiobiose,0.327637,0.382869,17.649437
6,D-Galactose,0.310722,0.344948,18.287066
7,D-Glucose,0.327523,0.363599,17.653758
8,Glycerol,0.376232,0.408531,20.817622
9,2-Keto-D-gluconic acid,0.275265,0.285054,14.623679


Now I will also make the predicted growth rates for each sugar anaerobically, and add it to the dataframe. Then in R i can make various plots.

In [19]:
pred_growth_anaer = [] #store the predicted growths here
pred_yld_anaer = []
with model:
    #set glc exchange to zero
    model.reactions.EX_glc__D_e.bounds = (0,1000)
    #set oxygen to 'anaerobic'
    model.reactions.EX_o2_e.bounds = (-0.005,1000)
    for carb in sugars:
        model_copy = copy.deepcopy(model) 
        met = model_copy.metabolites.get_by_id(carb)
        rct_id = 'EX_'+ carb
        exch = model_copy.reactions.get_by_id(rct_id)
        carbon = met.elements['C']
        lb = -30/carbon #i.e. how many mols of the sugar we would supply
        #set uptake
        exch.bounds = (lb,1000)
        #predict growth
        try: 
            growth = model_copy.optimize().objective_value
            MW = met.formula_weight/1000 #in g/mmol
            yld = (growth/-lb)/MW #in gx/gs
        except UserWarning:
            growth = 0
            yld = 0
        pred_growth_anaer.append(growth)
        pred_yld_anaer.append(yld)

c:\users\vivmol\appdata\local\continuum\anaconda3\envs\g-thermo-copy\lib\site-packages\cobra\util\solver.py:416: UserWarning:

solver status is 'infeasible'



In [24]:
#make small dataframe of sugars & anaerobic growth and then merge with the ordered one
df_anaerob = pd.DataFrame({'Sugar':sugar_name, 'Growth Anaerobic': pred_growth_anaer, 'Biomass yield (g/g) anaerobic':pred_yld_anaer})

In [25]:
df_tot = pd.merge(df_aer_ordered, df_anaerob, on='Sugar')

In [27]:
#export dataframe to visualize in R
df_tot.to_csv('../databases/Growth on various sugars.csv')

In [29]:
model.metabolites.prpp_c.summary()

In [31]:
model.metabolites.prbatp_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,ATPPRT,100.0,0.051793,atp_c + prpp_c --> ppi_c + prbatp_c
CONSUMING,PRATPP,100.0,0.051793,h2o_c + prbatp_c --> h_c + ppi_c + prbamp_c
